In [1]:
from model.model import ConvModel, SimCLR
import argparse
import torch
import torch.backends.cudnn as cudnn
from torchvision import models
from torchvision import transforms, datasets


In [15]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, ort, xla, lazy, vulkan, mps, meta, hpu, privateuseone device type at start of device string: gpu

In [10]:
composed_transforms = transforms.Compose([
      transforms.RandomResizedCrop(size=28),
      transforms.ElasticTransform(),
      transforms.RandomGrayscale(p=0.2),
      transforms.ToTensor()])

In [11]:
train_dataset = datasets.MNIST('./datasets', train=True,
                               transform=composed_transforms,
                               download=True)
g = torch.Generator()
g.manual_seed(0)

In [12]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=False,
    num_workers=12, pin_memory=True, drop_last=True, generator=g)

In [13]:
model = ConvModel(out_dim=10)
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)

C:\Users\kacpe\Desktop\GB\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kacpe\Desktop\GB\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
with torch.cuda.device(gpu_index):
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100)
    simclr.train(train_loader)

 66%|██████▌   | 155/234 [08:18<04:14,  3.22s/it]


KeyboardInterrupt: 

In [ ]:
for param in model.parameters():
    param.requires_grad = False